# OCR Common Error Table Generation Using Python difflib (New) 

Scarlett Hwang  
March 13th, 2020 - end date

In [27]:
import difflib as dl

In [195]:
conv = open("big_test/moby_full-chunk-bp-converted.txt", "r").read()
orig = open("big_test/moby_full-chunk-bp", "r").read()
#orig = "abcde cba abcde abcdex"
#conv = "abde ca ade ab ey"
# orig = "ever."
# conv = "ever ."
table = {}

In [183]:
print("| orig: ", orig, " \n| conv: ", conv)

| orig:  abcde cba abcde abcde  
| conv:  abde ca ade ab e


**chdict**: {'a' : {'a':100, 'b':1, 'Del':3}, 'Del' : {'i':3}}

In [199]:
# ndiff converted text in function of the original text
ndiff = dl.ndiff(orig, conv, linejunk = dl.IS_LINE_JUNK, charjunk = dl.IS_CHARACTER_JUNK)

# construct a list of each character
# each element is composed of three indices
# [0]: indicator (+ or - for errors, " " for correct conversion)
# [1]: space (ignore)
# [2]: character
list = [i for i in ndiff]  # list comprehension (dict, tuple, set comprehension)
list

['- \n',
 '  A',
 '  n',
 '  d',
 '   ',
 '  t',
 '  h',
 '  e',
 '   ',
 '  w',
 '  o',
 '  m',
 '  e',
 '  n',
 '   ',
 '  o',
 '  f',
 '   ',
 '  N',
 '  e',
 '  w',
 '   ',
 '  B',
 '  e',
 '  d',
 '- f',
 '+ i',
 '  o',
 '  r',
 '  d',
 '  ,',
 '   ',
 '  t',
 '  h',
 '  e',
 '  y',
 '   ',
 '  b',
 '  l',
 '  o',
 '  o',
 '  m',
 '   ',
 '+ t',
 '- l',
 '- i',
 '  k',
 '  e',
 '   ',
 '  t',
 '  h',
 '  e',
 '- i',
 '+ l',
 '  r',
 '   ',
 '  o',
 '  w',
 '  n',
 '   ',
 '  r',
 '  e',
 '  d',
 '   ',
 '  r',
 '  o',
 '  s',
 '  e',
 '  s',
 '  .',
 '   ',
 '  B',
 '  u',
 '  t',
 '  \n',
 '  r',
 '  o',
 '  s',
 '  e',
 '  s',
 '   ',
 '  o',
 '  n',
 '  l',
 '  y',
 '   ',
 '  b',
 '- l',
 '- o',
 '- o',
 '- m',
 '-  ',
 '  i',
 '+ o',
 '+ o',
 '+ m',
 '+  ',
 '+ i',
 '  n',
 '   ',
 '  s',
 '  u',
 '  m',
 '  m',
 '  e',
 '  r',
 '  ;',
 '   ',
 '  w',
 '  h',
 '  e',
 '  r',
 '  e',
 '  a',
 '  s',
 '   ',
 '  t',
 '  h',
 '  e',
 '   ',
 '  f',
 '  i',
 '  n',
 '  e',
 '   '

1. loop over this stuff
2. check the first character
3. if '-', followed by '+' _or_ '+' followed by '-': replacement

4. else if '-' -> deletion   
   chdict = table[ch]    
   chdict['del'] += 1
   table[ch] += chdict
   
   table[ch]['del'] += 1

5. else if '+' -> insertion
   chcount = table['del'][ch]    
   chcount += 1
   table['del'][ch] chcount


instead of dict['a']  use dict.get('a', 0)

In [180]:
# Very small test
# deletion passed
orig = "abcde"
conv = "absde"
ndiff = dl.ndiff(orig, conv)
list = [i for i in ndiff]
# table = {}
list

['  a', '  b', '- c', '+ s', '  d', '  e']

In [202]:
# construct initial dictionary with deletion option
# table["Del"] = {}
i = -1

# iterate through char array
while i < len(list) - 1:
    i += 1
    indicator = list[i][0]
    ch = list[i][2]
    
    # add char if not found in error table
    #if ch not in table:
        #table[ch] = {}
    
    # if original and converted chars match (successfully recognized)
    if indicator == " ":
        print(ch, "correct")
        #if ch not in table[ch]:
                #table[ch][ch] = 0
        chdict = table.get(ch, {})
        chdict[ch] = chdict.get(ch, 0) + 1
        table[ch] = chdict
        continue
    
    # if char is not the last character and +/- follows -/+ (substituted)
    if i + 1 < len(list): 
        ch1_ind = list[i + 1][0]
        ch1 = list[i + 1][2]
        if (indicator == "+" and ch1_ind == "-"): 
            print(ch, "replaced + -")
#             if ch not in list[i + 1][2]:
#                 table[list[i + 1][2]][ch] = 0 
#             table[list[i + 1][2]][ch] += 1
            chdict = table.get(ch1, {})
            chdict[ch] = chdict.get(ch, 0) + 1
            table[ch1] = chdict
            i += 1 # increment i and skip next char
            continue
        if (indicator == "-" and ch1_ind == "+"):
            print(ch, "replaced")
#             if list[i + 1][2] not in table[ch]:
#                 table[ch][list[i + 1][2]] = 0 
#             table[ch][list[i + 1][2]] += 1
            chdict = table.get(ch, {})
            chdict[ch1] = chdict.get(ch1, 0) + 1
            table[ch] = chdict
            i += 1 # increment i and skip next char
            continue
    
    # if char is not followed by another indicator and is deleted
    if indicator == "-":
        print(ch, "deleted")
#         if "Del" not in table[ch]:
#                 table[ch]["Del"] = 0 
#         table[ch]["Del"] += 1
        chdict = table.get(ch, {})
        chdict["Del"] = chdict.get("Del", 0) + 1
        table[ch] = chdict
        
    # if char is not followed by another indicator and is inserted 
    elif indicator == "+":
        print(ch, "inserted")
#         if ch not in table["Del"]:
#                 table["Del"][ch] = 0 
#         table["Del"][ch] += 1
        chdict = table.get("Del", {})
        chdict[ch] = chdict.get(ch, 0) + 1
        table["Del"] = chdict
    
    # pass any edge cases for now
    else:
        print("-- something weird going on ---", indicator, ch)
        pass

#table


 deleted
A correct
n correct
d correct
  correct
t correct
h correct
e correct
  correct
w correct
o correct
m correct
e correct
n correct
  correct
o correct
f correct
  correct
N correct
e correct
w correct
  correct
B correct
e correct
d correct
f replaced
o correct
r correct
d correct
, correct
  correct
t correct
h correct
e correct
y correct
  correct
b correct
l correct
o correct
o correct
m correct
  correct
t replaced + -
i deleted
k correct
e correct
  correct
t correct
h correct
e correct
i replaced
r correct
  correct
o correct
w correct
n correct
  correct
r correct
e correct
d correct
  correct
r correct
o correct
s correct
e correct
s correct
. correct
  correct
B correct
u correct
t correct

 correct
r correct
o correct
s correct
e correct
s correct
  correct
o correct
n correct
l correct
y correct
  correct
b correct
l deleted
o deleted
o deleted
m deleted
  deleted
i correct
o inserted
o inserted
m inserted
  inserted
i inserted
n correct
  correct
s correct
u correc

o deleted
n replaced
i inserted
l correct
e inserted
n inserted
c inserted
e inserted
  inserted
r inserted
e inserted
i inserted
g inserted
n inserted
e inserted
d inserted
, inserted
  inserted
o inserted
m inserted
l inserted
y correct
  correct
b correct
r correct
o correct
k correct
e correct
n correct
  correct
a correct
t deleted
  correct
t correct
i correct
m correct
e correct
s correct
  correct
b correct
y correct
  correct
t correct
h correct
e correct
  correct
s correct
h correct
r correct
i correct
e correct
k correct
s correct

 correct
o correct
f correct
  deleted
t correct
h correct
e correct
  correct
s correct
t correct
o correct
r correct
m correct
. correct
  correct
E correct
a correct
c correct
h correct
  correct
s correct
i correct
l correct
e correct
n correct
t correct
  correct
w correct
o correct
r correct
s correct
h correct
i correct
p correct
p correct
e correct
r correct
  correct
s correct
e correct
e correct
m correct
e correct
d correct
  correct
p

t correct
h correct
e correct
  correct
c correct
o correct
a correct
s correct
t correct
  correct
o correct
f correct
  correct
J correct
a correct
p correct
a correct
n correct
, correct
  correct
_ correct
A correct
u correct
g correct
u correct
s correct
t correct
_ correct

 correct
3 correct
_ correct
d correct
_ correct
, correct
  correct
1 correct
8 correct
3 correct
3 correct
. replaced
  correct
T correct
H correct
I correct
S correct
  correct
T correct
A correct
B correct
L correct
E correct
T correct
  correct
I correct
s correct
  correct
e correct
r correct
e correct
c correct
t correct
e correct
d correct
  correct
t correct
o correct
  correct
h correct
i correct
s correct
  correct
M correct
e correct
m correct
o correct
r correct
y correct
  correct
B correct
Y correct
  correct
H correct
I correct
S correct
  correct
W correct
I correct
D correct
O correct
W correct
. correct

 correct

 correct
S correct
h correct
a correct
k correct
i correct
n correct
g correct

### Tests

In [25]:

s1 = ['bacon\n', 'eggs\n', 'ham\n', 'guido\n']
s2 = ['python\n', 'eggy\n', 'hamster\n', 'guido\n']

diff = dl.ndiff(s1, s2)
print(''.join(diff), end="")

- bacon
+ python
- eggs
?    ^
+ eggy
?    ^
- ham
+ hamster
  guido


In [207]:
o = open("unittest/test3-orig.txt", "r").read()
c = open("unittest/test3-conv.txt", "r").read()

table = {}
n = dl.ndiff(o, c, linejunk = dl.IS_LINE_JUNK, charjunk = dl.IS_CHARACTER_JUNK)
list = [i for i in n]  # list comprehension (dict, tuple, set comprehension)

In [209]:
# construct initial dictionary with deletion option
# table["Del"] = {}
i = -1

# iterate through char array
while i < len(list) - 1:
    i += 1
    indicator = list[i][0]
    ch = list[i][2]
    
    # add char if not found in error table
    #if ch not in table:
        #table[ch] = {}
    
    # if original and converted chars match (successfully recognized)
    if indicator == " ":
        print(ch, "correct")
        #if ch not in table[ch]:
                #table[ch][ch] = 0
        chdict = table.get(ch, {})
        chdict[ch] = chdict.get(ch, 0) + 1
        table[ch] = chdict
        continue
    
    # if char is not the last character and +/- follows -/+ (substituted)
    if i + 1 < len(list): 
        ch1_ind = list[i + 1][0]
        ch1 = list[i + 1][2]
        if (indicator == "+" and ch1_ind == "-"): 
            print(ch, "replaced + -")
#             if ch not in list[i + 1][2]:
#                 table[list[i + 1][2]][ch] = 0 
#             table[list[i + 1][2]][ch] += 1
            chdict = table.get(ch1, {})
            chdict[ch] = chdict.get(ch, 0) + 1
            table[ch1] = chdict
            i += 1 # increment i and skip next char
            continue
        if (indicator == "-" and ch1_ind == "+"):
            print(ch, "replaced")
#             if list[i + 1][2] not in table[ch]:
#                 table[ch][list[i + 1][2]] = 0 
#             table[ch][list[i + 1][2]] += 1
            chdict = table.get(ch, {})
            chdict[ch1] = chdict.get(ch1, 0) + 1
            table[ch] = chdict
            i += 1 # increment i and skip next char
            continue
    
    # if char is not followed by another indicator and is deleted
    if indicator == "-":
        print(ch, "deleted")
#         if "Del" not in table[ch]:
#                 table[ch]["Del"] = 0 
#         table[ch]["Del"] += 1
        chdict = table.get(ch, {})
        chdict["Del"] = chdict.get("Del", 0) + 1
        table[ch] = chdict
        
    # if char is not followed by another indicator and is inserted 
    elif indicator == "+":
        print(ch, "inserted")
#         if ch not in table["Del"]:
#                 table["Del"][ch] = 0 
#         table["Del"][ch] += 1
        chdict = table.get("Del", {})
        chdict[ch] = chdict.get(ch, 0) + 1
        table["Del"] = chdict
    
    # pass any edge cases for now
    else:
        print("-- something weird going on ---", indicator, ch)
        pass

table

s correct
a correct
n correct
d correct
s correct
, replaced + -

 deleted

 correct

 correct
C correct
H correct
A correct
P correct
T correct
E correct
R correct
  correct
7 correct
. correct
  correct
T correct
h correct
e correct
  correct
C correct
h correct
a correct
p correct
e correct
l correct
. deleted

 correct

 correct
I correct
n correct
  correct
t correct
h correct
i correct
s correct
  correct
s correct
a correct
m correct
e correct
  correct
N correct
e correct
w correct
  correct
B correct
e correct
d correct
f replaced
o correct
r correct
d correct
  correct
t correct
h correct
e correct
r correct
e correct
  correct
s correct
t correct
a correct
n correct
d correct
s correct
  correct
a correct
  correct
W correct
h correct
a correct
l correct
e correct
m correct
a correct
n correct
’ replaced
s correct
  correct
C correct
h correct
a correct
p correct
e correct
l correct
, correct
  correct
a correct
n correct
d correct
  correct
f correct
e correct
w correct
  c

{'s': {'s': 24},
 'a': {'a': 38},
 'n': {'n': 22, 'Del': 2},
 'd': {'d': 18, 'Del': 4},
 '.': {',': 2, '.': 4, 'Del': 2, 'i': 2},
 '\n': {'Del': 2, '\n': 12},
 'C': {'C': 6},
 'H': {'H': 2},
 'A': {'A': 2},
 'P': {'P': 4},
 'T': {'T': 4},
 'E': {'E': 2},
 'R': {'R': 2},
 ' ': {' ': 72, 'Del': 10},
 '7': {'7': 2},
 'h': {'h': 26},
 'e': {'e': 38},
 'p': {'p': 6},
 'l': {'l': 10},
 'I': {'I': 4, 'L': 2, 'Del': 2},
 't': {'t': 24, 'Del': 4},
 'i': {'i': 16, 'Del': 2},
 'm': {'m': 12},
 'N': {'N': 2},
 'w': {'w': 6},
 'B': {'B': 2},
 'f': {'i': 2, 'f': 10},
 'o': {'o': 20, 'Del': 4},
 'r': {'r': 16},
 'W': {'W': 2},
 '’': {"'": 2},
 ',': {',': 6},
 'y': {'y': 6},
 'b': {'b': 2},
 'u': {'u': 6},
 'O': {'O': 2},
 'c': {'c': 6},
 'k': {'k': 2},
 'S': {'S': 2},
 'v': {'v': 2},
 'Del': {'a': 2, 'd': 2, ' ': 2, 'n': 2, 'o': 2, 't': 2}}